In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from scipy.ndimage.measurements import label
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

%matplotlib inline

In [ ]:
class boxe_target():
    def __init__(self, coord, weight):
        self.coordinates_history = []
        self.actual_coordinates = coord #((x1, y1), (x2, y2))
        self.centroid = ((coord[0][0] + coord[1][0]) / 2, (coord[0][1] + coord[1][1]) / 2)
        self.best_coordinates = None
        self.updated = False
        self.unupdated_sc = 0
        self.weight = weight
        
    def compute_best_coord(self):
        self.best_coordinates = np.mean(self.coordinates_history[-10:], axis=0)
        
    def get_best_coord(self):
        #print("self.coordinates_history : {}".format(self.coordinates_history))
        if len(self.coordinates_history) < 2 :
            #print("self.actual_coordinates : {}".format(self.actual_coordinates))
            return None
        elif self.weight < 8:
            return None
        else:
            self.compute_best_coord()
            #print("self.best_coodinate : {}".format(self.best_coordinates))
            return self.best_coordinates
        
class boxe_target_col():
    def __init__(self):
        self.boxe_list = []
        self.first_iter = True
        
    def add(self, target):
        to_add = True
        for element in self.boxe_list:
            distance = np.sqrt(np.power(target.centroid[0] - element.centroid[0], 2) +
                      np.power(target.centroid[1] - element.centroid[1], 2))
            if distance < 50:
                element.coordinates_history.append(target.actual_coordinates)
                element.weight += target.weight
                element.updated = True
                element.unupdated_sc = 0
                to_add = False
                break
        
        if to_add:
            target
            self.boxe_list.append(target)
    
    def refresh(self):
        print("len(self.boxe_list) before : {}".format(len(self.boxe_list)))
        for elem in self.boxe_list:
            if not elem.updated:
                elem.unupdated_sc += 1
        
        if not self.first_iter:
            self.boxe_list = [e for e in self.boxe_list if e.weight > 2]
            
        for elem in self.boxe_list:
            elem.updated = False
            elem.weight -= 3 * elem.unupdated_sc
        
        self.first_iter = False
        print("len(self.boxe_list) after : {}".format(len(self.boxe_list)))
        
        

In [ ]:
def convert_color(to_convert, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(to_convert, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(to_convert, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(to_convert, cv2.COLOR_RGB2LUV)

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, visualise=vis,
                                  feature_vector=feature_vec, block_norm='L2-Hys')
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, visualise=vis,
                       feature_vector=feature_vec, block_norm='L2-Hys')
        return features

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [ ]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        file_features = single_img_features(image, color_space=color_space, spatial_size=spatial_size,
                        hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                        spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat)
        
        features.append(file_features)
    # Return list of feature vectors
    return features

In [ ]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [ ]:
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

In [ ]:
# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2,
                        spatial_feat=True, hist_feat=True, hog_feat=True):   
    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
        
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        #ctrans_tosearch = convert_color(feature_image, conv='RGB2YCrCb')
        ch1 = feature_image[:,:,0]
        ch2 = feature_image[:,:,1]
        ch3 = feature_image[:,:,2]
        # Compute individual channel HOG features for the entire image
        hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog_features = np.hstack((hog1, hog2, hog3))
        #8) Append features to list
        img_features.append(hog_features.ravel())
        
        
    

    #9) Return concatenated array of features
    return np.concatenate(img_features)

In [ ]:
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

In [ ]:
cars = glob.glob('./vehicles/*/*.png')
notcars = glob.glob('./non-vehicles/*/*.png')

print(len(cars))
print(len(notcars))

In [ ]:
# Just for fun choose random car / not-car indices and plot example images
car_ind = np.random.randint(0, len(cars))
notcar_ind = np.random.randint(0, len(notcars))

def data_look(car_list, notcar_list):
    data_dict = {}
    # Define a key in data_dict "n_cars" and store the number of car images
    data_dict["n_cars"] = len(car_list)
    # Define a key "n_notcars" and store the number of notcar images
    data_dict["n_notcars"] = len(notcar_list)
    # Read in a test image, either car or notcar
    example_img = mpimg.imread(car_list[0])
    # Define a key "image_shape" and store the test image shape 3-tuple
    data_dict["image_shape"] = example_img.shape
    # Define a key "data_type" and store the data type of the test image.
    data_dict["data_type"] = example_img.dtype
    # Return data_dict
    return data_dict

def plot_data_look(cars, notcars):
    data_info = data_look(cars, notcars)

    print('Your function returned a count of',
          data_info["n_cars"], ' cars and',
          data_info["n_notcars"], ' non-cars')
    print('of size: ', data_info["image_shape"], ' and data type:',
          data_info["data_type"])

    # Read in car / not-car images
    car_image = mpimg.imread(cars[car_ind])
    notcar_image = mpimg.imread(notcars[notcar_ind])

    # Plot the examples
    fig = plt.figure()
    plt.subplot(121)
    plt.imshow(car_image)
    plt.title('Example Car Image')
    plt.subplot(122)
    plt.imshow(notcar_image)
    plt.title('Example Not-car Image')
    
plot_data_look(cars, notcars)

In [ ]:
def plot_hog_feature():
    # Read in car / not-car images
    car_image = mpimg.imread(cars[car_ind])
    feature_image = cv2.cvtColor(car_image, cv2.COLOR_RGB2YCrCb)
    ch1 = feature_image[:,:,0]
    ch2 = feature_image[:,:,1]
    ch3 = feature_image[:,:,2]
    # Compute individual channel HOG features for the entire image
    hog1, hog_car1 = get_hog_features(ch1, 9, 8, 2, feature_vec=False, vis=True)
    hog2, hog_car2 = get_hog_features(ch2, 9, 8, 2, feature_vec=False, vis=True)
    hog3, hog_car3 = get_hog_features(ch3, 9, 8, 2, feature_vec=False, vis=True)
    
    fig = plt.figure()
    plt.subplot(131)
    plt.imshow(hog_car1)
    plt.title('Example hog_car1')
    plt.subplot(132)
    plt.imshow(hog_car2)
    plt.title('Example hog_car2')
    plt.subplot(133)
    plt.imshow(hog_car3)
    plt.title('Example hog_car3')
    
plot_hog_feature()

In [ ]:
colorspace = 'YCrCb'
orient = 9
spatial = 32
histbin = 32
pix_per_cell = 8
cell_per_block = 2

In [ ]:
def stack_array_scale(cars, notcars):
    
    car_features = extract_features(cars, color_space=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=2,
                        spatial_feat=True, hist_feat=True, hog_feat=True)
    
    notcar_features = extract_features(notcars, color_space=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=2,
                        spatial_feat=True, hist_feat=True, hog_feat=True)
    
    # Create an array stack of feature vectors
    X = np.vstack((car_features, notcar_features)).astype(np.float64)
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)

    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    
    return scaled_X, X_scaler

In [ ]:
def generate_dataset(cars, notcars):

    # Apply the scaler to X
    scaled_X, X_scaler = stack_array_scale(cars, notcars)

    # Define the labels vector
    y = np.hstack((np.ones(len(cars)), np.zeros(len(notcars))))

    scaled_X, y = shuffle(scaled_X, y)
    
    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)
    
    return X_train, X_test, y_train, y_test, X_scaler

X_train, X_test, y_train, y_test, X_scaler = generate_dataset(cars, notcars)

In [ ]:

print('Using spatial binning of:', spatial,
      'and', histbin, 'histogram bins')
print('Feature vector length:', len(X_train[0]))

'''t = time.time()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svr = SVC()
clf = GridSearchCV(svr, parameters)
clf.fit(X_train, y_train)
t2 = time.time()
print(round(t2 - t, 2), 'Seconds to train SVC...')'''

#svc = LinearSVC()
svc = RandomForestClassifier(max_depth=18, random_state=0)
# Check the training time for the SVC
t = time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2 - t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t = time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these', n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2 - t, 5), 'Seconds to predict', n_predict, 'labels with SVC')

In [ ]:
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient,
              pix_per_cell, cell_per_block, spatial_size, hist_bins, window):
    window_list = []
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
    #ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YUV')
    ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YCrCb)
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            img_features = []
            
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (window,window))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)
            
            img_features.append(spatial_features)
            img_features.append(hist_features)
            img_features.append(hog_features) 

            # Scale features and make a prediction np.concatenate(img_features)
            #np.concatenate((spatial_features, hist_features, hog_features)).reshape(1, -1)
            res = np.concatenate(img_features).reshape(1, -1)
                        
            test_features = X_scaler.transform(res)    
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))  
            test_prediction = svc.predict(test_features)
            
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                window_list.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)))
                
    return window_list

In [ ]:
def find_cars_multiple_scale(image, met=[1, 2]):
    hot_windows = []
    if 1 in met:
        size_w = [200, 128, 96, 64]
        y_ss = [[500, 656], [400, 550], [400, 500], [400, 500]]

        windows = []
        for i in range(4):
            tmp = slide_window(image, x_start_stop=[None, None], y_start_stop=y_ss[i], 
                        xy_window=(size_w[i], size_w[i]), xy_overlap=(0.5, 0.5))

            windows.extend(tmp)

        hot_windows_tmp = search_windows(image, windows, svc, X_scaler, color_space=colorspace, 
                            spatial_size=(spatial, spatial), hist_bins=histbin, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=2, spatial_feat=True, 
                            hist_feat=True, hog_feat=True) 
        hot_windows.extend(hot_windows_tmp)
    #3.5,        3.5,         2,          
    #[464, 660], [400, 596], [432, 560],
    if 2 in met:
        scales = [ 2,        1.5,
                  1.5,        1,           1,        0.7]
        y_ss = [ [380, 528], [380, 496],
                [415, 542], [410, 470], [400, 480], [400, 480]]
        for i in range(len(scales)):
            boxes = find_cars(image, y_ss[i][0], y_ss[i][1], scales[i], svc, X_scaler, orient=orient,
                         pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                              spatial_size=(32, 32), hist_bins=histbin, window = 64)

            hot_windows.extend(boxes)
    return hot_windows

In [ ]:
def plot_find_cars(image):
#    boxes = find_cars(image, 400, 656, 1.5, svc, X_scaler, orient=9,
#                 pix_per_cell=8, cell_per_block=2, spatial_size=(32, 32), hist_bins=32, window = 64)
    
    hot_windows = find_cars_multiple_scale(image, met=[2])                      

    
    img_boxes = draw_boxes(image, hot_windows, color=(0, 0, 255), thick=6)
    
    plt.imshow(img_boxes)
    
image = mpimg.imread('./test_images/test1.jpg')
plot_find_cars(image)

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels, collection_boxes, heatmap):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        
        weight = max(heatmap[bbox[0][1]:bbox[1][1], bbox[0][0]:bbox[1][0]].ravel())
        
        #print("bbox[1][0] = x{}".format(bbox[1][0]))
        #print("bbox[1][1] = y{}".format(bbox[1][1]))
        #print("bbox[0][0] = x{}".format(bbox[0][0]))
        #print("bbox[0][1] = y{}".format(bbox[0][1]))
        #print(heatmap.shape)
        
        target = boxe_target(bbox, weight)
        collection_boxes.add(target)
    
    for element in collection_boxes.boxe_list:
        # Draw the box on the image
        temp = element.get_best_coord()
        if temp != None:
            cv2.rectangle(img, (int(temp[0][0]), int(temp[0][1])),
                      (int(temp[1][0]), int(temp[1][1])), (0,0,element.weight), 6)
    
    collection_boxes.refresh()
    # Return the image
    return img

def heat_pipeline(boxes):
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Add heat to each box in box list
    heat = add_heat(heat, boxes)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat, 1)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    return label(heatmap), heatmap
    
def plot_heat_pipeline(image, collection_boxes):
    #boxes = find_cars(image, 400, 656, 1.5, svc, X_scaler, orient=9,
    #             pix_per_cell=8, cell_per_block=2, spatial_size=(32, 32), hist_bins=32)
       
    hot_windows = find_cars_multiple_scale(image, met=[2])   

    labels, heatmap = heat_pipeline(hot_windows)
    draw_img = draw_labeled_bboxes(np.copy(image), labels, collection_boxes, heatmap)

    fig = plt.figure()
    plt.subplot(121)
    plt.imshow(draw_img)
    plt.title('Car Positions')
    plt.subplot(122)
    plt.imshow(heatmap)
    plt.title('Heat Map')
    fig.tight_layout()

image = mpimg.imread('./test_images/test6.jpg')
collection_boxes = boxe_target_col()
plot_heat_pipeline(image, collection_boxes)

In [ ]:
collection_boxes = boxe_target_col()
def process_image(image):
    hot_windows = find_cars_multiple_scale(image, met=[2])   

    labels, heatmap = heat_pipeline(hot_windows)
    draw_img = draw_labeled_bboxes(np.copy(image), labels, collection_boxes, heatmap)
    return draw_img

In [ ]:
def global_pipeline_video(input_file, output):
    ## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
    ## To do so add .subclip(start_second,end_second) to the end of the line below
    ## Where start_second and end_second are integer values representing the start and end of the subclip
    ## You may also uncomment the following line for a subclip of the first 5 seconds
    ##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
    clip1 = VideoFileClip(input_file)
    white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
    %time white_clip.write_videofile(output, audio=False)

global_pipeline_video("./project_video.mp4", "./project_video_processed_met2.mp4")